# Quiz 3 - Laboratorio de modelos y simulaciones.
### Juan Esteban Pineda Lopera
### C.C. 1001248691

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.signal as signal

In [ ]:
df = pd.read_csv('experimento.csv')

print(df.shape)
df.head()

In [ ]:
t = df.t
u = df.u
y = df.y
uv = df.uv
yv = df.yv
N = len(y)               # Número de datos
p = round(0.6*N)
Ts = np.mean(np.diff(t)) # Periodo de muestréo

# Predicción para 'y' y 'u'

## Desconvolución

In [ ]:
ypad = np.concatenate([y, np.zeros(round(p))])

h_desc, residuo = signal.deconvolve(ypad, u)
y_desc = np.convolve(h_desc, u)

## Mínimo error cuadrático

In [ ]:
def minimo_error_cuadratico(T, u, y, p, N):
  UU = np.zeros((N, p))

  # Se llena la matriz sobre sus columnas
  for i in range(p):
    if i == 0:
      UU[:,0] = u

    else:
      UU[i:, i] = u[:N-i]

  # Se calcula la formula
  UU *= T
  AA = UU.T @ UU                  # (U'U)
  b = UU.T @ y                    # U'y
  h = np.linalg.solve(AA, b) * T  # [(U'U)^(-1)U'y] * T

  return h

In [ ]:
h_lce = minimo_error_cuadratico(Ts, u, y, p, N)

y_lce = np.convolve(h_lce, u)

plt.figure()
plt.plot(h_lce)
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(t, y, color='black', linewidth=8, label='Real')
plt.plot(t, y_desc[0:len(t)], color='red', linewidth=4, label='Predicha por desconvolución')
plt.plot(t, y_lce[0:len(t)], color='blue', linewidth=2, label='Predicha por mínimo error cuadrático')

plt.xlim(0, np.max(t))
plt.grid()
plt.legend()
plt.xlabel('Tiempo')
plt.ylabel('Valores de salida')
plt.title("Predicciones por desconvolución y mínimo error cuadrático para los datos de 'y' y 'u'")

plt.tight_layout()
plt.show()

# Predicción para 'yv' y 'vu'

In [ ]:
yv_desc = np.convolve(h_desc, uv)

In [ ]:
yv_lce = np.convolve(h_lce, uv)

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(t, yv, color='black', linewidth=8, label='Real')
plt.plot(t, yv_desc[0:len(t)], color='red', linewidth=4, label='Predicha por desconvolución')
plt.plot(t, yv_lce[0:len(t)], color='blue', linewidth=2, label='Predicha por mínimo error cuadrático')

plt.xlim(0, np.max(t))
plt.grid()
plt.legend()
plt.xlabel('Tiempo')
plt.ylabel('Valores de salida')
plt.title("Predicciones por desconvolución y mínimo error cuadrático para los datos de 'yv' y 'vu'")

plt.tight_layout()
plt.show()

# Preguntas.

## ¿Por qué es necesario usar datos adicionales en el proceso  de identificación del sistema?

Podría ser el caso que el sistema simulado sea coherente solo para ciertos valores de entrada, pero que con otras, difiera en cierta medida del módelo real.

El mínimo error cuadrático, por ejemplo, actúa para minimizar dicho error entre las salidas y entradas dados. Al depender estas técnicas de los datos especificados originalmente, puede ser el caso de que se encuentren modelos que predigan aceptablemente las salidas sin ser exactamente iguales al sistema original, lo que provoca que, al ingresar valores diferentes de los usados en la construcción de la simulación, los resultados ya no sean los esperados.

Al utilizar datos adicionales ajenos al proceso de construcción, es posible asegurar o descartar el módelo como buen predictor del sistema original, pues es posible concluir que su rendimiento no se debe a su adaptación a los datos de entrenamiento.

## ¿Qué metricas usaría para revisar el éxito en la identificación y cómo las interpretaría?

Podría calculares el error de predicción, así como el mínimo error cuadrático. Ambas métricas comparan las salidas esperadas con las salidas obtenidas, para informar sobre que tan alejados se encuentran ambos vectores. En ambos casos, se espera que un valor menor corresponda a aquel modelo que aproxime mejor los valores reales esperados, siendo cero un indicador de que el modelo construido y el sistema predicho son exactamente iguales. No se espera que la identificación arroje un modelo idéntico, pero se puede establecer un margen de error aceptable, de manera que se puede establecer el exito o no dependiendo de dicho margen.

Otra posibilidad es la correspondancia, que compara que tan similares son los datos predichos de los datos reales. En un módelo idéntico al sistema real, se espera que la correspondancia sea máxima, por lo que es posible medir el éxito de la identifación midiendo que tan correspondientes son estos datos.